In [25]:
import requests 
from datetime import datetime, timedelta
import pandas as pd
from pandas.errors import EmptyDataError
import csv
import os
# import pyspark.pandas as ps


## Funções

In [13]:
def busca_artigo_keyword(url, chave_api, keyword):
    url_nova = url + '?q='+keyword+'&apiKey='+chave_api
    r = requests.get(url_nova)
    data = r.json()
    return data

def busca_artigo_keyword_publicacao_hora(url_api, api_key, palavra_chave):
    agora = datetime.utcnow()
    hora_anterior = agora - timedelta(hours=1)
    hora_anterior_formatada = hora_anterior.strftime('%Y-%m-%dT%H:%M:%SZ')
    agora_formatada = agora.strftime('%Y-%m-%dT%H:%M:%SZ')
    url_nova = f"{url_api}?q={palavra_chave}&apiKey={api_key}&from={hora_anterior_formatada}&to={agora_formatada}"
    resposta = requests.get(url_nova)
    dados_json = resposta.json()
    return dados_json

## Extract inicial (histórico)
Traz todos os artigos com a palavra chave 'bitcoin'

In [4]:
chave_api = '657a3d08bfd94920bc9e527fac57ee28'
url = 'https://newsapi.org/v2/everything'

dados = busca_artigo_keyword(url, chave_api, 'bitcoin')
dados['articles'][0]


{'source': {'id': 'wired', 'name': 'Wired'},
 'author': 'Joel Khalili',
 'title': 'What’s Behind the Bitcoin Price Surge? Vibes, Mostly',
 'description': 'The price of bitcoin has climbed to a new all-time high. But assigning the cryptocurrency a value is anything but trivial.',
 'url': 'https://www.wired.com/story/bitcoin-price-record-economics/',
 'urlToImage': 'https://media.wired.com/photos/65ef46042ca08b0e59a9682f/191:100/w_1280,c_limit/031124-business-bitcoin-economics.jpg',
 'publishedAt': '2024-03-12T18:13:56Z',
 'content': 'The latest surge in the price of bitcoin is increasing the clamor around it, says Dal Bianco, drawing in yet more speculators and creating a self-reinforcing cycle. Likewise, when collective confiden… [+2967 chars]'}

Cria CSV com a primeira chamada, enviando o histórico das noticias

In [5]:
articles = dados['articles']
df = pd.DataFrame(articles)
caminho_arquivo_csv = 'dados.csv'
df.to_csv(caminho_arquivo_csv, index=False)


## Extract de hora em hora
Traz todos os artigos com a palavra chave 'bitcoin' que foram publicados na ultima hora

In [20]:
dados_ultima_hora = busca_artigo_keyword_publicacao_hora(url, chave_api, 'bitcoin')
artigos_ultima_hora = dados_ultima_hora['articles']
artigos_ultima_hora

[]

In [23]:
# Adicionar alguma logica para rodar de hora em hora

In [22]:
if artigos_ultima_hora:
    try:
        df = pd.read_csv('dados_hoje.csv')
    except (FileNotFoundError, EmptyDataError):
        df = pd.DataFrame()

    novos_dados = pd.DataFrame(artigos_ultima_hora)
    df = pd.concat([df, novos_dados], ignore_index=True)

    df.to_csv('dados_hoje.csv', index=False)
else:
    print('Nenhum dado encontrado para adicionar ao CSV.')

Nenhum dado encontrado para adicionar ao CSV.


## Extract final do dia

In [24]:
# Adicionar alguma logica para rodar no fim do dia

In [26]:
arquivo_dados_hoje = 'dados_hoje.csv'
arquivo_dados = 'dados.csv'

if os.path.isfile(arquivo_dados_hoje):
    df_hoje = pd.read_csv(arquivo_dados_hoje)

    if os.path.isfile(arquivo_dados):
        df_dados = pd.read_csv(arquivo_dados)
        df_resultado = pd.concat([df_dados, df_hoje], ignore_index=True)
        df_resultado.to_csv(arquivo_dados, index=False)
    else:
        os.rename(arquivo_dados_hoje, arquivo_dados)
else:
    print('O arquivo dados_hoje.csv não foi encontrado.')

O arquivo dados_hoje.csv não foi encontrado.
